In [4]:
import requests 
import json
import pandas as pd

#choose keyword, subreddit, and date to be analyzed
q = 'smtm' 
subreddit='khiphop' 
after = '2021-10-01' 
before = '2021-12-10' 


def getPushshiftData(subreddit=None, before=None, after=None, ids=None, q=None):
    suffix=''
    if (before is not None):
        suffix += '&before='+str(before)
    if (after is not None):
        suffix += '&after='+str(after)
    if (subreddit is not None):
        suffix += '&subreddit='+subreddit
    if (q is not None):
        suffix += '&q='+str(q)

    url = 'https://api.pushshift.io/reddit/search/comment/?sort=desc&size=1500'+suffix
    print('loading '+url)
    r = requests.get(url)
    data = json.loads(r.content)
    if len(data['data']) > 0:
        prev_end_date = data['data'][-1]['created_utc']
    else:
        prev_end_date = None
    return (data, prev_end_date)

(comments_tmp, prev_end_date) = getPushshiftData(subreddit=subreddit, q=q, after=after, before=before)
comments = comments_tmp['data']
while prev_end_date is not None:
    (comments_tmp, prev_end_date) = getPushshiftData(subreddit=subreddit, q=q, before=prev_end_date-1, after=after)
    if prev_end_date is not None:
        comments.extend(comments_tmp['data'])

loading https://api.pushshift.io/reddit/search/comment/?sort=desc&size=1500&before=2021-12-10&after=2021-10-01&subreddit=khiphop&q=smtm
loading https://api.pushshift.io/reddit/search/comment/?sort=desc&size=1500&before=1638165777&after=2021-10-01&subreddit=khiphop&q=smtm
loading https://api.pushshift.io/reddit/search/comment/?sort=desc&size=1500&before=1637197751&after=2021-10-01&subreddit=khiphop&q=smtm
loading https://api.pushshift.io/reddit/search/comment/?sort=desc&size=1500&before=1634364738&after=2021-10-01&subreddit=khiphop&q=smtm
loading https://api.pushshift.io/reddit/search/comment/?sort=desc&size=1500&before=1633053584&after=2021-10-01&subreddit=khiphop&q=smtm


In [5]:
table_df = pd.DataFrame(comments)
# preview all the data not yet cleaned
table_df.head(5)

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_patreon_flair,author_premium,awarders,body,collapsed_because_crowd_control,collapsed_reason_code,comment_type,created_utc,gildings,id,is_submitter,link_id,locked,no_follow,parent_id,permalink,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,unrepliable_reason,archived,body_sha1,can_gild,collapsed,collapsed_reason,controversiality,distinguished,gilded,retrieved_utc,score_hidden,subreddit_name_prefixed,subreddit_type,author_cakeday
0,[],None,luvzz12,None,None,[],None,None,None,text,t2_8adwws14,False,False,[],But the reason he blew up at all was due to hi...,None,None,None,1639049670,{},hnua28n,False,t3_rb3tgk,False,False,t1_hnturqv,/r/khiphop/comments/rb3tgk/in_your_opinion_whi...,1.642881e+09,4,True,False,khiphop,t5_2vuc3,None,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],None,dyang44,None,None,[],None,None,None,text,t2_54syw,False,False,[],I thought he was going for mainstream appeal i...,None,None,None,1638997444,{},hnrmi90,False,t3_rbn55j,False,False,t1_hnpqwbx,/r/khiphop/comments/rbn55j/gaeko_changmo_don_m...,1.642553e+09,10,True,False,khiphop,t5_2vuc3,None,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],None,Alias-Eberhardt,None,None,[],None,None,None,text,t2_55jhcbfc,False,False,[],"Again, as I said: we don’t know, and we can’t ...",None,None,None,1638996236,{},hnrj9xl,False,t3_rb7ysn,False,True,t3_rb7ysn,/r/khiphop/comments/rb7ysn/is_smtm_rigged/hnrj...,1.642541e+09,1,True,False,khiphop,t5_2vuc3,None,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],None,TurnedUpTurtle,None,None,[],None,None,None,text,t2_3v00qowi,False,False,[],Your completely missing the point. nowhere hav...,None,None,None,1638975540,{},hnq233t,False,t3_rb7ysn,False,True,t1_hnq11jz,/r/khiphop/comments/rb7ysn/is_smtm_rigged/hnq2...,1.642354e+09,1,True,False,khiphop,t5_2vuc3,None,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],None,Oneew,None,None,[],None,None,None,text,t2_z378i,False,False,[],"Okay, why couldn't we see this side of BE'O on...",None,None,None,1638970185,{},hnpqwbx,False,t3_rbn55j,False,False,t3_rbn55j,/r/khiphop/comments/rbn55j/gaeko_changmo_don_m...,1.642316e+09,23,True,False,khiphop,t5_2vuc3,None,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#get row and column number
table_df.shape

(320, 50)

In [7]:
#turn data into csv file
table_df.to_csv('./smtm_comments.csv', header=True, index=False, columns=list(table_df.axes[1]))

In [8]:
#check to see which column to be used
khiphop = pd.read_csv("/content/smtm_comments.csv", index_col=0)
khiphop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 320 entries, [] to []
Data columns (total 49 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   associated_award                 0 non-null      float64
 1   author                           320 non-null    object 
 2   author_flair_background_color    0 non-null      float64
 3   author_flair_css_class           30 non-null     object 
 4   author_flair_richtext            320 non-null    object 
 5   author_flair_template_id         30 non-null     object 
 6   author_flair_text                30 non-null     object 
 7   author_flair_text_color          36 non-null     object 
 8   author_flair_type                320 non-null    object 
 9   author_fullname                  320 non-null    object 
 10  author_patreon_flair             320 non-null    bool   
 11  author_premium                   320 non-null    bool   
 12  awarders                   

In [9]:
import nltk, re, pprint
from nltk import word_tokenize

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
#since there are too many irrelevant data in the dataframe, instead of using df.drop() function i will keep only the 'body' column.
newdf = pd.DataFrame(table_df, columns=['body'])
newdf.head(5)

,body
0,But the reason he blew up at all was due to hi...
1,I thought he was going for mainstream appeal i...
2,"Again, as I said: we don’t know, and we can’t ..."
3,Your completely missing the point. nowhere hav...
4,"Okay, why couldn't we see this side of BE'O on..."


In [12]:
def clean(text):
# Removes all special characters and numericals leaving only the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the body column
newdf['cleaned_body'] = newdf['body'].apply(clean)
newdf.head()

,body,cleaned_body
0,But the reason he blew up at all was due to hi...,But the reason he blew up at all was due to hi...
1,I thought he was going for mainstream appeal i...,I thought he was going for mainstream appeal i...
2,"Again, as I said: we don’t know, and we can’t ...",Again as I said we don t know and we can t kno...
3,Your completely missing the point. nowhere hav...,Your completely missing the point nowhere have...
4,"Okay, why couldn't we see this side of BE'O on...",Okay why couldn t we see this side of BE O on ...


In [15]:
def tokenization_s(sentences): 
    s_new = []
    for sent in (sentences[:][0]): #For NumpY = sentences[:]
        s_token = sent_tokenize(sent)
        if s_token != '':
            s_new.append(s_token)
    return s_new

In [14]:
 newdftokenized = newdf['cleaned_body']
#then to tokenize

 newdftokenized = newdftokenized.apply(word_tokenize)
 newdftokenized.head()
 #Im done here but I lost the table format. 

0    [But, the, reason, he, blew, up, at, all, was,...
1    [I, thought, he, was, going, for, mainstream, ...
2    [Again, as, I, said, we, don, t, know, and, we...
3    [Your, completely, missing, the, point, nowher...
4    [Okay, why, couldn, t, we, see, this, side, of...
Name: cleaned_body, dtype: object

In [19]:
#write lemmatize function
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()
def lemmatization(words):
    new = []
    lem_words = [lemmatizer.lemmatize(x) for x in (words[:][0])]
    new.append(lem_words)
    return new

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
#move on to NRC
with open("/content/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt","r") as f:
  lexicontext = f.readlines()
wordline = []
for line in lexicontext:
  wordline.append(line.split("\t"))

In [23]:
newlex = {}
for entry in wordline:
  if entry[1] == "positive":
    if int(entry[2][0]) > 0:
      newlex[entry[0]] = 1
  elif entry[1] == "negative":
    if int(entry[2][0]) > 0:
      newlex[entry[0]] = -1